In [ ]:
%pip install --upgrade scapy

#### **Этап 1. Изучение Scapy**
1. Изучите основы работы с Scapy.
2. Настройте Scapy для перехвата HTTP-трафика, используйте скрипт scapy для отправки HTTP-запросов.

In [ ]:
import logging
from datetime import datetime
from scapy.all import sniff, load_layer
from scapy.layers.http import HTTPRequest, HTTPResponse
from scapy.packet import Raw

In [ ]:
# Загружаем слой HTTP для корректного разбора пакетов Scapy
load_layer("http")

# Настройка логирования
file_log = logging.FileHandler(f'{datetime.now().date()}_log.log')
console_out = logging.StreamHandler()
    
logging.basicConfig(handlers=(file_log, console_out), 
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S',
                        level=logging.INFO)
logger = logging.getLogger(__name__)

# Список типичных сигнатур XSS для поиска в трафике (Этап 3, Примеры XSS) [cite: 38, 39, 40]
XSS_PAYLOADS = [
    "<script>alert('XSS')</script>",
    "<img src=\"nonexistent.jpg\" onerror=\"alert('XSS')\">",
    "alert('XSS')",
    "javascript:alert"
]

In [ ]:
def _check_xss_signature(content: str) -> bool:
    """
    Проверяет вхождение известных XSS-сигнатур в контент.
    
    Args:
        content (str): Анализируемая строка (тело запроса или ответа).
    
    Returns:
        bool: True, если найден XSS паттерн.
    """
    for payload in XSS_PAYLOADS:
        # Проверяем наличие пейлоада или его URL-кодированных вариаций (упрощенно)
        if payload in content:
            return True
    return False

In [ ]:
def _safe_decode(byte_data: bytes) -> str:
    """
    Безопасно декодирует байты в строку, игнорируя ошибки.
    """
    try:
        return byte_data.decode('utf-8', errors='ignore')
    except Exception:
        return str(byte_data)

In [ ]:
def _analyze_request(packet):
    """
    Анализирует HTTP-запрос. Выводит метод, хост и путь.
    Ищет попытки инъекции XSS в отправляемых данных.
    
    Args:
        packet: Объект пакета Scapy с слоем HTTPRequest.
    """
    url = packet[HTTPRequest].Host.decode() + packet[HTTPRequest].Path.decode()
    method = packet[HTTPRequest].Method.decode()
    
    log_msg = f"[>>] REQUEST: {method} {url}"
    
    # Проверка наличия сырых данных (тело запроса, параметры)
    if packet.haslayer(Raw):
        load = _safe_decode(packet[Raw].load)
        if _check_xss_signature(load):
            log_msg += f"\n    [!] ВНИМАНИЕ: Обнаружен XSS-пейлоад в запросе:\n    {load}"
            
    logger.info(log_msg)

In [ ]:
def _analyze_response(packet):
    """
    Анализирует HTTP-ответ. Выводит статус.
    Ищет следы отраженной (Reflected) XSS атаки в ответе сервера.
    Это необходимо для выполнения Этапа 4[cite: 42, 43].
    
    Args:
        packet: Объект пакета Scapy с слоем HTTPResponse.
    """
    status = packet[HTTPResponse].Status_Code.decode() if packet[HTTPResponse].Status_Code else "Unknown"
    
    log_msg = f"[<<] RESPONSE: Status {status}"

    if packet.haslayer(Raw):
        load = _safe_decode(packet[Raw].load)
        # Если пейлоад найден в ответе, значит сервер вернул его без санитизации (успешная XSS)
        if _check_xss_signature(load):
            log_msg += f"\n    [!!!] КРИТИЧНО: Обнаружено отражение XSS в ответе (Уязвимость подтверждена):\n    {load[:200]}..." 
            
    logger.info(log_msg)

In [ ]:
def process_packet(packet):
    """
    Callback-функция для обработки каждого перехваченного пакета.
    Анализирует HTTP запросы и ответы на наличие подозрительных данных.
    
    Args:
        packet: Объект пакета Scapy.
    """
    # Проверяем наличие слоя HTTPRequest (Исходящий трафик)
    if packet.haslayer(HTTPRequest):
        _analyze_request(packet)

    # Проверяем наличие слоя HTTPResponse (Входящий трафик)
    elif packet.haslayer(HTTPResponse):
        _analyze_response(packet)

In [ ]:
def start_sniffer(interface: str = None):
    """
    Запускает сниффер Scapy.
    
    Args:
        interface (str): Название сетевого интерфейса (например, 'eth0', 'Wi-Fi').
                         Если None, Scapy выберет интерфейс по умолчанию.
    """
    print(f"[*] Запуск перехвата HTTP трафика на интерфейсе: {interface or 'Default'}")
    print("[*] Нажмите Ctrl+C для остановки.")
    
    # Фильтр 'tcp port 80' важен, так как Google Gruyere обычно работает по HTTP (не HTTPS)
    # Это соответствует заданию по сбору трафика[cite: 31].
    sniff(
        iface=interface,
        filter="tcp port 80",
        prn=process_packet,
        store=False
    )

In [ ]:
try:
    start_sniffer()
except KeyboardInterrupt:
    print("\n[*] Перехват остановлен пользователем.")